In [8]:
import numpy as np

# Define the reward matrix
rewards = np.array([
    [0, 0.2],
    [0, 0.2],
    [0, 0.2],
    [0, 0.2],
    [1, 0.2]
])

# Define transition probabilities for a0 and a1
transition_a0 = np.array([
    [0, 0.8, 0.2, 0, 0],
    [0, 0, 0.8, 0.2, 0],
    [0, 0, 0.2, 0.8, 0],
    [0, 0, 0, 0, 1],
    [0, 0, 0, 0, 1]
])

transition_a1 = np.array([
    [0.9, 0.1, 0, 0, 0],
    [0.9, 0.1, 0, 0, 0],
    [0.9, 0, 0.1, 0, 0],
    [0.9, 0, 0.1, 0, 0],
    [0.9, 0, 0.1, 0, 0]
])

transition = np.array([transition_a0, transition_a1])
print(transition.shape)

# Discount factor
gamma = 0.95

# Number of states and actions
num_states = 5
num_actions = 2

(2, 5, 5)


In [15]:
import collections
trans = collections.defaultdict(dict)
for state in range(num_states):
  for action in range(num_actions):
    trans[state][action] = []
    for nxt_state, prob in enumerate(transition[action, state, :]):
      if prob == 0:
        continue
      else:
        trans[state][action].append((nxt_state, prob))

print(trans)

defaultdict(<class 'dict'>, {0: {0: [(1, 0.8), (2, 0.2)], 1: [(0, 0.9), (1, 0.1)]}, 1: {0: [(2, 0.8), (3, 0.2)], 1: [(0, 0.9), (1, 0.1)]}, 2: {0: [(2, 0.2), (3, 0.8)], 1: [(0, 0.9), (2, 0.1)]}, 3: {0: [(4, 1.0)], 1: [(0, 0.9), (2, 0.1)]}, 4: {0: [(4, 1.0)], 1: [(0, 0.9), (2, 0.1)]}})


#Optimal Q Values for all state-action pairs using dynamic programming:

In [9]:
# Value Iteration Function
def value_iteration(rewards
                    , transition_a0
                    , transition_a1
                    , gamma
                    , num_states
                    , num_actions
                    , threshold=0.00000000001):
    Q = np.zeros((num_states, num_actions))
    delta = float('inf')

    while delta > threshold:
        delta = 0
        for s in range(num_states):
            for a in range(num_actions):
                old_value = Q[s, a]
                if a == 0:
                    Q[s, a] = rewards[s, a] + gamma * np.sum(transition_a0[s, :] * np.max(Q, axis=1))
                else:
                    Q[s, a] = rewards[s, a] + gamma * np.sum(transition_a1[s, :] * np.max(Q, axis=1))
                delta = max(delta, abs(old_value - Q[s, a]))

    return Q

# Compute the Q-values using Value Iteration
Q_values = value_iteration(rewards, transition_a0, transition_a1, gamma, num_states, num_actions)
print("Q-values from Value Iteration:\n", Q_values)




Q-values from Value Iteration:
 [[16.42772839 15.87577877]
 [17.15864198 15.87577877]
 [17.82716049 15.93928802]
 [19.         15.93928802]
 [20.         15.93928802]]


#Q learning with epsilon greedy exploration:

In [24]:
import numpy as np

# Initialization
num_states = 5
num_actions = 2
Q = np.zeros((num_states, num_actions))  # Q-value matrix
lr = 0.01
epsilon = 0.1  # Initial exploration probability
min_epsilon = 0.001  # Minimum value of epsilon
epsilon_decay = 0.999  # Decay factor for epsilon
gamma = 0.95  # Discount factor
episodes = 500  # Total number of episodes
steps_per_episode = 1000  # Number of steps per episode

# Q-Learning with Epsilon-Greedy Exploration
for epi in range(episodes):
    state = 0  # Always start from state s0
    for step in range(steps_per_episode):
        if np.random.rand() < epsilon:
            action = np.random.choice(num_actions)  # Explore: choose a random action
        else:
            action = np.argmax(Q[state])  # Exploit: choose the best action based on Q-values

        # Simulate transition based on action
        tran = trans[state][action]
        reward = rewards[state][action]

        # print(tran)

        q_sa = reward
        for nxt_state, prob in tran:
          q_sa += prob * gamma * np.max(Q[nxt_state])

        # Q-value update
        td_error = q_sa - Q[state][action]
        Q[state][action] += lr * td_error

        # Transition to next state
        nxt_states = [nxt_state for nxt_state, prob in tran]
        nxt_probs = [prob for nxt_state, prob in tran]

        state = np.random.choice(nxt_states, 1, p=nxt_probs)[0]

    # Epsilon decay
    # epsilon = max(min_epsilon, epsilon * epsilon_decay)
    if (epi + 1) % 50 == 0:
        print(f'Episode: {epi + 1}, Q(s,a) = \n{Q}')

# Print the final Q-values
print("Final Q-values:")
print(Q)

Episode: 50, Q(s,a) = 
[[16.40997478  7.10373645]
 [17.15107778  6.93260958]
 [17.8260686   7.58182233]
 [18.99991805  6.87460271]
 [19.99999089 15.89577355]]
Episode: 100, Q(s,a) = 
[[16.42772839 12.79096806]
 [17.15864197 11.95274927]
 [17.82716049 13.63009513]
 [19.         13.25215336]
 [20.         15.93928802]]
Episode: 150, Q(s,a) = 
[[16.4277284  14.68844722]
 [17.15864198 14.4398231 ]
 [17.82716049 15.21960247]
 [19.         15.01326868]
 [20.         15.93928802]]
Episode: 200, Q(s,a) = 
[[16.4277284  15.47070154]
 [17.15864198 15.328612  ]
 [17.82716049 15.71499005]
 [19.         15.66757291]
 [20.         15.93928802]]
Episode: 250, Q(s,a) = 
[[16.4277284  15.75450659]
 [17.15864198 15.63090893]
 [17.82716049 15.85635841]
 [19.         15.85374961]
 [20.         15.93928802]]
Episode: 300, Q(s,a) = 
[[16.4277284  15.83760117]
 [17.15864198 15.78152933]
 [17.82716049 15.91771924]
 [19.         15.91493457]
 [20.         15.93928802]]
Episode: 350, Q(s,a) = 
[[16.4277284  15.

#Compare your results obtained by the two methods:

The Q values for all state-action pairs using dynamic programming results were:
```
Q-values from Value Iteration:
 [[16.42772839 15.87577877]
 [17.15864198 15.87577877]
 [17.82716049 15.93928802]
 [19.         15.93928802]
 [20.         15.93928802]]
```
Meanwhile Q learning with epsilon greedy exploration, the values were:
```
Final Q-values:
[[16.4277284  15.8754464 ]
 [17.15864198 15.87346848]
 [17.82716049 15.93916614]
 [19.         15.93900426]
 [20.         15.93928802]]
```
As you can tell, they are incredibly close to each other.

